In [1]:
from typing import Tuple, Dict, Any
from preprocessing_utils import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
from torch.optim import Adam
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import random
import wandb
import torch.nn as nn
import torch.nn.functional as F



device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")

#user_item_path = '/Volumes/DeepLearner/Search & Recommendation System/Data/australian_users_items_clean.json'
device


device(type='cuda', index=0)

In [2]:
user_item_path = '/notebooks/australian_users_items_clean.json'

In [3]:
def create_mappings(df, column):
    unique_ids = sorted(df[column].unique())  # Sorting added here
    print(f"Number of unique {column}: {len(unique_ids)}")
    id_to_idx = {id_: idx for idx, id_ in enumerate(unique_ids)}
    return id_to_idx

class SimplifiedSteamDataset(Dataset):
    def __init__(self, user_item_df, user_mapping, item_mapping):
        self.df = user_item_df.copy()
        
        self.df['user_idx'] = self.df['user_id'].map(user_mapping)
        self.df['item_idx'] = self.df['item_id'].map(item_mapping)
        
        print(f"Missing user_idx: {self.df['user_idx'].isna().sum()}")
        print(f"Missing item_idx: {self.df['item_idx'].isna().sum()}")
        
        # Scaling playtime
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.y = self.scaler.fit_transform(self.df['playtime_forever'].values.reshape(-1, 1)).flatten()
        
        self.user_idxs = self.df['user_idx'].values
        self.item_idxs = self.df['item_idx'].values
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return self.y[idx], self.user_idxs[idx], self.item_idxs[idx]

In [4]:
def prepare_simple_data_loaders(user_item_path, batch_size=32):
    # Load data
    user_item_df = load_json_to_df(user_item_path)
    user_item_df.sort_values(by=['user_id', 'playtime_forever'], ascending=[True, False], inplace=True)
    
    # Drop duplicates based on 'user_id' and 'item_id'
    #user_item_df.drop_duplicates(subset=['user_id', 'item_id'], inplace=True)
    def test_mapping(mapping, unique_ids):
        # Test 1: Ensure every unique ID has a mapping
        for unique_id in unique_ids:
            assert unique_id in mapping, f"{unique_id} not found in mapping"

        # Test 2: Ensure reverse mapping is consistent
        reverse_mapping = {v: k for k, v in mapping.items()}
        for unique_id in unique_ids:
            idx = mapping[unique_id]
            assert reverse_mapping[idx] == unique_id, f"Inconsistent mapping for {unique_id}"
    
    # Create mappings
    user_mapping = create_mappings(user_item_df, 'user_id')
    item_mapping = create_mappings(user_item_df, 'item_id')

    test_mapping(user_mapping, user_item_df['user_id'].unique())
    test_mapping(item_mapping, user_item_df['item_id'].unique())

    # Create the final dataset using the new mappings
    full_dataset = SimplifiedSteamDataset(user_item_df, user_mapping, item_mapping)

    # Split into train, validation, and test sets
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, test_size], 
    generator=torch.Generator().manual_seed(42)  # set the seed for reproducibility
    )

# Create a DataFrame from val_dataset
    val_data = [val_dataset[i] for i in range(len(val_dataset))]
    val_df = pd.DataFrame(val_data, columns=['playtime_scaled', 'user_idx', 'item_idx'])
    # Create DataLoader for each set
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Extract the indices from val_dataset
    val_indices = val_dataset.indices

    # Create a validation DataFrame using these indices
    val_df = full_dataset.df.iloc[val_indices].reset_index(drop=True)

    return train_loader, val_loader, test_loader, user_mapping, item_mapping, full_dataset.df, val_df


In [5]:
train_loader, val_loader, test_loader, user_mapping, item_mapping, user_item_df, val_df = prepare_simple_data_loaders(user_item_path)

Number of unique user_id: 70834
Number of unique item_id: 10978
Missing user_idx: 0
Missing item_idx: 0


In [6]:
def get_actual_top_k_games(user_id, user_item_df, user_mapping, k=5):
    
    user_data = user_item_df[user_item_df['user_idx'] == user_id]
    top_k_actual = user_data.sort_values(by='playtime_forever', ascending=False).head(k)
    return top_k_actual['item_name'].tolist()

def get_predicted_top_k_games(model, user_id, user_mapping, item_mapping, user_item_df, k=5):
    model.eval()
    
    user_idx = torch.LongTensor([user_mapping[user_id]] * len(item_mapping)).to(device)    
    # Convert item_mapping values to a tensor and move to the same device as the model
    all_item_idxs = torch.LongTensor(list(item_mapping.values())).to(device)

    # Make predictions
    with torch.no_grad():
        predictions = model(user_idx, all_item_idxs).cpu().numpy().flatten()

    # Extract top-k items
    top_k_indices = predictions.argsort()[-k:][::-1]
    top_k_item_idxs = [list(item_mapping.values())[i] for i in top_k_indices]

    top_k_item_names = [user_item_df.loc[user_item_df['item_idx'] == idx, 'item_name'].iloc[0] for idx in top_k_item_idxs]

    return top_k_item_names

def precision_at_k(y_true, y_pred, k, threshold=0.5):
    # Sort by predicted score and take top k
    top_k_indices = np.argsort(y_pred)[::-1][:k]
    top_k_true = y_true[top_k_indices]
    
    # Count number of true positives in top k (playtime above threshold)
    true_positives = np.sum(top_k_true > threshold)
    
    return true_positives / k

def predict_for_random_user(model, user_mapping, item_mapping, user_item_df, k=5):
    # Randomly select a user ID
    random_user_id = random.choice(list(user_mapping.keys()))
    print(random_user_id)
    print(f"Making predictions for random user {random_user_id}")
    # Check if the random_user_id exists in the DataFrame
    print(random_user_id in user_item_df['user_id'].values)

    
    # Call the existing function to make predictions for this user
    top_k_games = get_predicted_top_k_games(model, random_user_id, user_mapping, item_mapping, user_item_df, k)
    
    return random_user_id, top_k_games

def recall_at_k(y_true, y_pred, k, threshold=0.5):
    # Sort by predicted score and take top k
    top_k_indices = np.argsort(y_pred)[::-1][:k]
    top_k_true = y_true[top_k_indices]
    
    # Count number of true positives in top k (playtime above threshold)
    true_positives = np.sum(top_k_true > threshold)
    
    # Count the total number of actual positives (relevant items)
    total_actual_positives = np.sum(y_true > threshold)
    
    if total_actual_positives == 0:
        return 0
    
    return true_positives / total_actual_positives


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NCF(nn.Module):
    def __init__(self, num_users, num_items, embed_dim, layers=[16, 8], dropout=False):
        super(NCF, self).__init__()
        
        self.num_users = num_users
        self.num_items = num_items
        self.embed_dim = embed_dim
        
        # GMF embeddings
        self.user_gmf_embedding = nn.Embedding(num_users, embed_dim)
        self.item_gmf_embedding = nn.Embedding(num_items, embed_dim)
        
        # MLP embeddings
        self.user_mlp_embedding = nn.Embedding(num_users, layers[0] // 2)
        self.item_mlp_embedding = nn.Embedding(num_items, layers[0] // 2)
        
        # MLP layers
        mlp_modules = []
        for i, (in_size, out_size) in enumerate(zip(layers[:-1], layers[1:])):
            mlp_modules.append(nn.Linear(in_size, out_size))
            mlp_modules.append(nn.ReLU())
            if dropout:
                mlp_modules.append(nn.Dropout(p=0.2))
        self.mlp_layers = nn.Sequential(*mlp_modules)
        
        # Final prediction layer
        self.prediction_layer = nn.Linear(layers[-1] + embed_dim, 1)
        
    def forward(self, user_indices, item_indices):
        user_gmf_embed = self.user_gmf_embedding(user_indices)
        item_gmf_embed = self.item_gmf_embedding(item_indices)
        
        user_mlp_embed = self.user_mlp_embedding(user_indices)
        item_mlp_embed = self.item_mlp_embedding(item_indices)
        
        # GMF part
        gmf_product = torch.mul(user_gmf_embed, item_gmf_embed)
        
        # MLP part
        mlp_input = torch.cat([user_mlp_embed, item_mlp_embed], dim=1)
        mlp_output = self.mlp_layers(mlp_input)
        
        # Final layer
        final_input = torch.cat([gmf_product, mlp_output], dim=1)
        prediction = self.prediction_layer(final_input)
        
        return prediction.squeeze(-1)


In [8]:
# Initialize wandb
wandb.init(project="Steam Rec Engine_vNew_200dim", config={"learning_rate": 0.001})

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
import wandb
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader


wandb.config.update({
    "learning_rate": 0.001,
    "batch_size": 32,
    "embed_dim": 200,
    "epochs": 20,
})

#wandb.config.update({"Model Architecture": str(model)})

# Initialize NCF model, optimizer, and loss function
# Assuming you have defined NCF class as discussed
model = NCF(num_users=len(user_mapping), num_items=len(item_mapping), embed_dim=200)

wandb.config.update({"Model Architecture": str(model)})

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)


#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)


# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Number of epochs and interval to save model
n_epochs = 20
save_interval = 5  # Save every 2 epochs

for epoch in range(1, n_epochs + 1):
    # Training loop
    model.train()
    train_loss = 0.0
    for batch_idx, batch_data in enumerate(tqdm(train_loader, desc=f"Training epoch {epoch}")):
        y_batch, user_ids, item_ids = batch_data
        y_batch = y_batch.float()
        user_ids, item_ids = user_ids.long(), item_ids.long()
        y_batch = y_batch.to(device)
        user_ids, item_ids = user_ids.to(device), item_ids.to(device)
        optimizer.zero_grad()
        outputs = model(user_ids, item_ids)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        
        wandb.log({"Batch Train Loss": loss.item()}, step=batch_idx)
        #scheduler.step()

        #wandb.log({"Batch Train Loss": loss.item()})
        train_loss += loss.item()
    
    for name, param in model.named_parameters():
        if param.requires_grad:
            wandb.log({f"Gradients/{name}": wandb.Histogram(param.grad.cpu().numpy())})
    #scheduler.step()
    # Inside your training loop
    wandb.log({"Learning Rate": scheduler.optimizer.param_groups[0]['lr']})
    train_loss /= len(train_loader)
    wandb.log({"Train Loss": train_loss})
    print(f"Epoch {epoch} - Training loss: {train_loss}")

    # Validation loop
    model.eval()
    y_true_val = []
    y_pred_val = []
    val_loss = 0.0
    with torch.no_grad():
        for batch_data in tqdm(val_loader, desc=f"Validating epoch {epoch}"):
            y_batch, user_ids, item_ids = batch_data
            
            y_batch = y_batch.float()

            user_ids, item_ids = user_ids.long(), item_ids.long()

            y_batch = y_batch.to(device)

            user_ids, item_ids = user_ids.to(device), item_ids.to(device)

            outputs = model(user_ids, item_ids)

            loss = criterion(outputs.squeeze(), y_batch)

            batch_rmse = sqrt(mean_squared_error(y_batch.cpu().numpy(), outputs.cpu().numpy()))
            wandb.log({"Batch Validation RMSE": batch_rmse})


            val_loss += loss.item()
            y_true_val.extend(y_batch.cpu().numpy())
            y_pred_val.extend(outputs.cpu().numpy())

    val_loss /= len(val_loader)
    scheduler.step(val_loss)
    val_rmse = sqrt(mean_squared_error(y_true_val, y_pred_val))
    val_precision_at_k = precision_at_k(np.array(y_true_val), np.array(y_pred_val), k=5)
    wandb.log({"Validation Loss": val_loss, "Validation RMSE": val_rmse, "Validation Precision@5": val_precision_at_k})
    print(f"Epoch {epoch} - Validation loss: {val_loss}, RMSE: {val_rmse}, Precision@5: {val_precision_at_k}")

    # Log example users
    table_rows = []
    for _ in range(5):
        random_user_id, random_top_k_games = predict_for_random_user(model, user_mapping, item_mapping, user_item_df, k=10)
        actual_games = get_actual_top_k_games(user_mapping[random_user_id], user_item_df, user_mapping, k=10)
        print(f"Actual games: {actual_games}")
        print(f"Predicted games: {random_top_k_games}")
        # Create a row for this user
        row = [random_user_id, ", ".join(actual_games), ", ".join(random_top_k_games)]
        
        # Add this row to the table
        table_rows.append(row)
        
    table = wandb.Table(data=table_rows, columns=["User ID", "Actual Top-5 Games", "Predicted Top-5 Games"])

    wandb.log({f"Example Users Epoch {epoch}": table})
    
    # Save the model at intervals
    if epoch % save_interval == 0:
        torch.save(model.state_dict(), f"model_epoch_{epoch}.pt")
        wandb.save(f"model_epoch_{epoch}.pt")

Training epoch 1: 100%|██████████| 112511/112511 [16:00<00:00, 117.16it/s]


Epoch 1 - Training loss: 0.0004021331169123971


Validating epoch 1: 100%|██████████| 24110/24110 [00:31<00:00, 754.28it/s]


Epoch 1 - Validation loss: 0.00013815338514148288, RMSE: 0.011753929748983908, Precision@5: 0.0
babypanay
Making predictions for random user babypanay
True
Actual games: ['Age of Mythology: Extended Edition', "Garry's Mod", 'The Elder Scrolls V: Skyrim', 'Age of Empires II: HD Edition', 'STAR WARS™ Empire at War: Gold Pack', 'Far Cry® 3', "Sid Meier's Civilization V", 'Grand Theft Auto IV', 'Need for Speed: Hot Pursuit', 'Terraria']
Predicted games: ['Samantha Swift and the Golden Touch', 'Mimic Arena', 'Metro Last Light JP', '4 Elements', 'Nurse Love Addiction', 'Prominence', 'Run Rabbit Run', 'Unnamed Fiasco', 'Planetbase', 'MotoGP™13']
76561198050010300
Making predictions for random user 76561198050010300
True
Actual games: ['Total War: SHOGUN 2', 'The Sims(TM) 3', 'Farming Simulator 2011', 'Empire: Total War', 'Napoleon: Total War', 'Microsoft Flight', 'Theatre of War 2: Africa 1943', 'Demolition Company Gold', 'Medieval II: Total War', "Garry's Mod"]
Predicted games: ['Valiant', "

Training epoch 2: 100%|██████████| 112511/112511 [15:44<00:00, 119.07it/s]


...

Training epoch 16: 100%|██████████| 112511/112511 [15:45<00:00, 118.97it/s]


Epoch 16 - Training loss: 6.697981898334734e-05


Validating epoch 16: 100%|██████████| 24110/24110 [00:32<00:00, 744.07it/s]


Epoch 16 - Validation loss: 6.968202279669933e-05, RMSE: 0.008347678152879203, Precision@5: 0.0
76561198059098764
Making predictions for random user 76561198059098764
True
Actual games: ['Robocraft', 'Prison Architect', "Garry's Mod", 'Age of Empires Online', 'Unturned', 'PAYDAY 2', 'Left 4 Dead 2', 'Total War: ROME II - Emperor Edition', 'Warhammer 40,000: Dawn of War – Soulstorm', 'PlanetSide 2']
Predicted games: ['Molly', 'Cubicolor', 'Rugby Challenge 2', 'Bravada', 'DreamDesk VR Beta', 'Occurrence at JCR Outpost', 'Miko Gakkou: Second Year', "Dark Parables: The Little Mermaid and the Purple Tide Collector's Edition", 'Champions Of Chaos 2', 'TransOcean 2: Rivals']
76561198089058865
Making predictions for random user 76561198089058865
True
Actual games: ['Dota 2 Test', 'Afterfall InSanity - Dirty Arena Edition']
Predicted games: ['DreamDesk VR Beta', 'Web Designer 10', 'Mystik Belle', 'The Ables: Freepoint High', 'Zen Fish SIM', 'Supreme: Pizza Empire', 'Isbarah', 'City Quest', 'Chr

Training epoch 17:  72%|███████▏  | 81029/112511 [11:22<04:24, 119.12it/s]

In [ ]:
user_mapping

   val_loss /= len(val_loader)
    val_rmse = sqrt(mean_squared_error(y_true_val, y_pred_val))
    val_precision_at_k = precision_at_k(np.array(y_true_val), np.array(y_pred_val), k=5)
    
    print(f"Epoch {epoch} - Validation loss: {val_loss}, RMSE: {val_rmse}, Precision@5: {val_precision_at_k}")

    # Display top-k games for a sample user (let's say the first user in our dataset)
    #sample_user_id = list(user_mapping.keys())[0]

    random_user_id, random_top_k_games = predict_for_random_user(model, user_mapping, item_mapping, user_item_df, k=5)
    print(f"Actual top 5 games for user {random_user_id}: {get_actual_top_k_games(user_mapping[random_user_id], user_item_df, user_mapping, k=5)}")
    print(f"Predicted top 5 games for user {random_user_id}: {random_top_k_games}")